## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
tf.enable_eager_execution()

### Collect Data

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('/gdrive/My Drive/AIML/Lab_Internal_R6/prices.csv')

### Check all columns in the dataset

In [6]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date', 'symbol'], axis=1, inplace=True)

In [8]:
data.columns

Index([u'open', u'close', u'low', u'high', u'volume'], dtype='object')

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [10]:
X = data.loc[0:999,['open','close','low','high']]
len(X)

1000

In [0]:
y = data.loc[0:999,'volume']


In [12]:
#Divide volume by 1000000 as  the values are very high
y = y/1000000
len(y)

1000

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=123)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = np.array(y_train.astype('float32'))
y_test = np.array(y_test.astype('float32'))

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
#y_train = transformer.fit_transform(y_train)
X_test = transformer.transform(X_test)
#y_test = transformer.transform(y_test)

In [18]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((700, 4), (300, 4), (700,), (300,))


## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [23]:
#Train for 100 Steps
# Observe the training loss for every iteration
for i in range(100):
    
    w, b = train(X_train, y_train, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(y_train, prediction(X_train, w, b)).numpy())
          #loss(y_train, prediction(X_train, w, b)))

('Current Loss on iteration', 0, 175.58798)
('Current Loss on iteration', 1, 173.73335)
('Current Loss on iteration', 2, 172.02417)
('Current Loss on iteration', 3, 170.4489)
('Current Loss on iteration', 4, 168.99712)
('Current Loss on iteration', 5, 167.65913)
('Current Loss on iteration', 6, 166.42609)
('Current Loss on iteration', 7, 165.2897)
('Current Loss on iteration', 8, 164.24245)
('Current Loss on iteration', 9, 163.27739)
('Current Loss on iteration', 10, 162.38782)
('Current Loss on iteration', 11, 161.56793)
('Current Loss on iteration', 12, 160.8125)
('Current Loss on iteration', 13, 160.11623)
('Current Loss on iteration', 14, 159.47455)
('Current Loss on iteration', 15, 158.8832)
('Current Loss on iteration', 16, 158.33817)
('Current Loss on iteration', 17, 157.83585)
('Current Loss on iteration', 18, 157.37297)
('Current Loss on iteration', 19, 156.94649)
('Current Loss on iteration', 20, 156.55307)
('Current Loss on iteration', 21, 156.19072)
('Current Loss on iterat

In [24]:
#Train for 100 Steps
#Observe Training loss at every 5th iteration
for i in range(100):
    
    w, b = train(X_train, y_train, w, b, learning_rate=0.01)
    if i%5==0 :
      print('Current Loss on iteration', i, 
            loss(y_train, prediction(X_train, w, b)).numpy())
            #loss(y_train, prediction(X_train, w, b)))

('Current Loss on iteration', 0, 151.93793)
('Current Loss on iteration', 5, 151.9356)
('Current Loss on iteration', 10, 151.93416)
('Current Loss on iteration', 15, 151.93317)
('Current Loss on iteration', 20, 151.93251)
('Current Loss on iteration', 25, 151.93208)
('Current Loss on iteration', 30, 151.93176)
('Current Loss on iteration', 35, 151.93158)
('Current Loss on iteration', 40, 151.9314)
('Current Loss on iteration', 45, 151.93134)
('Current Loss on iteration', 50, 151.93121)
('Current Loss on iteration', 55, 151.93123)
('Current Loss on iteration', 60, 151.93114)
('Current Loss on iteration', 65, 151.93117)
('Current Loss on iteration', 70, 151.93124)
('Current Loss on iteration', 75, 151.93117)
('Current Loss on iteration', 80, 151.93121)
('Current Loss on iteration', 85, 151.93114)
('Current Loss on iteration', 90, 151.9311)
('Current Loss on iteration', 95, 151.93118)


### Get the shapes and values of W and b

In [25]:
print(w.shape)
print(b.shape)

(4, 1)
(1,)


In [26]:
print("Value of w:\n")
print w.numpy()

Value of w:

[[1.2644618]
 [1.2690701]
 [1.251506 ]
 [1.2798462]]


In [27]:
print("Value of b:\n")
print b.numpy()

Value of b:

[2.532586]


### Model Prediction on 1st Examples in Test Dataset

In [28]:
y_predict = prediction(X_test[0:1],w,b)
print("Predicted Values:\n")
print(y_predict.numpy())

Predicted Values:

[[5.0650673]]


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [29]:
!ls -l '/gdrive/My Drive/AIML/Lab_Internal_R6'

total 50515
-rw------- 1 root root     5107 Jan 12 09:02 11_Iris.csv
-rw------- 1 root root 51667341 Jan 12 09:02 prices.csv
-rw------- 1 root root    54696 Jan 12 12:13 R6_Internal_Lab_UpdatedTF2_Prices_Iris-3.ipynb


In [0]:
iris = pd.read_csv('/gdrive/My Drive/AIML/Lab_Internal_R6/11_Iris.csv')

In [31]:
iris.columns

Index([u'Id', u'SepalLengthCm', u'SepalWidthCm', u'PetalLengthCm',
       u'PetalWidthCm', u'Species'],
      dtype='object')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
iris = pd.get_dummies(iris, columns=['Species'])
iris.drop('Id', axis=1, inplace=True)

In [33]:
iris.columns

Index([u'SepalLengthCm', u'SepalWidthCm', u'PetalLengthCm', u'PetalWidthCm',
       u'Species_Iris-setosa', u'Species_Iris-versicolor',
       u'Species_Iris-virginica'],
      dtype='object')

### Splitting the data into feature set and target set

In [34]:
X = iris[['SepalLengthCm','SepalWidthCm','PetalLengthCm', 'PetalWidthCm']]
X.columns

Index([u'SepalLengthCm', u'SepalWidthCm', u'PetalLengthCm', u'PetalWidthCm'], dtype='object')

In [35]:
y = iris[['Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica']]
y.columns

Index([u'Species_Iris-setosa', u'Species_Iris-versicolor',
       u'Species_Iris-virginica'],
      dtype='object')

In [36]:
X.shape, y.shape

((150, 4), (150, 3))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=123)

In [0]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [0]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3,input_shape=(4,), activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Model Training 

In [41]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=100,
          batch_size=X_train.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/100


W0112 12:13:57.636499 140718256146304 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/api/_v1/estimator/__init__.py:12: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



105/105 [==============================] - 1s 9ms/sample - loss: 4.2426 - acc: 0.3810 - val_loss: 5.0298 - val_acc: 0.2222
Epoch 2/100
105/105 [==============================] - 0s 118us/sample - loss: 3.9256 - acc: 0.3810 - val_loss: 4.6551 - val_acc: 0.2222
Epoch 3/100
105/105 [==============================] - 0s 127us/sample - loss: 3.6303 - acc: 0.3810 - val_loss: 4.3169 - val_acc: 0.2222
Epoch 4/100
105/105 [==============================] - 0s 90us/sample - loss: 3.3676 - acc: 0.3810 - val_loss: 4.0274 - val_acc: 0.2222
Epoch 5/100
105/105 [==============================] - 0s 75us/sample - loss: 3.1483 - acc: 0.3810 - val_loss: 3.7932 - val_acc: 0.2222
Epoch 6/100
105/105 [==============================] - 0s 84us/sample - loss: 2.9771 - acc: 0.3810 - val_loss: 3.6102 - val_acc: 0.2222
Epoch 7/100
105/105 [==============================] - 0s 73us/sample - loss: 2.8491 - acc: 0.3714 - val_loss: 3.4670 - val_acc: 0.2667
Epoch 8/100
105/105 [==============================] - 0s 8

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.get_weights()

[array([[-0.10925541, -0.59646183,  0.033714  ],
        [-0.307147  ,  0.4449178 , -0.58272606],
        [ 0.48906133,  0.7655457 ,  0.17696875],
        [-0.99374664, -0.75979406,  0.44017068]], dtype=float32),
 array([ 0.15785375, -0.1002849 , -0.05756884], dtype=float32)]

### Model Prediction

In [44]:
model.predict(X_test)

array([[0.24096973, 0.31199792, 0.44703233],
       [0.15598029, 0.31317002, 0.5308497 ],
       [0.14268507, 0.31518814, 0.5421268 ],
       [0.22533171, 0.4571437 , 0.31752458],
       [0.37998042, 0.4301751 , 0.18984447],
       [0.17440538, 0.3971997 , 0.4283949 ],
       [0.27428806, 0.25392994, 0.47178197],
       [0.32634866, 0.4922305 , 0.18142083],
       [0.35427493, 0.4383786 , 0.20734636],
       [0.25659835, 0.4293966 , 0.31400508],
       [0.12495258, 0.24168463, 0.6333628 ],
       [0.2938239 , 0.56552094, 0.14065519],
       [0.2136101 , 0.40705094, 0.37933904],
       [0.18915011, 0.40974444, 0.40110537],
       [0.14539865, 0.23291922, 0.62168217],
       [0.14262791, 0.36861703, 0.48875514],
       [0.36970234, 0.40885943, 0.22143824],
       [0.35798907, 0.4508337 , 0.19117725],
       [0.23411657, 0.41726428, 0.34861907],
       [0.3546556 , 0.39856717, 0.24677728],
       [0.3581347 , 0.43711066, 0.20475468],
       [0.2315193 , 0.39698747, 0.37149322],
       [0.

### Save the Model

In [0]:
model.save('iris.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?